# Neo4j Runway End to End Example

## Introduction

## Load Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
from neo4j_runway import Discovery, GraphDataModeler, IngestionGenerator, LLM, PyIngest


## Load Data
For this experiment we will use the record of bank failures in the United States. Thsi data is published by the Federal Deposit Insurance Corporation (FDIC) and is available at https://www.fdic.gov/bank/individual/failed/banklist.csv

In [70]:
failedBank_df = pd.read_csv('https://www.fdic.gov/bank/individual/failed/banklist.csv', encoding='ISO-8859-1')
failedBank_df.shape

(569, 7)

## Preprocess Data
All features in the data need to be strings.

In [71]:
# trim whitepace from column names
failedBank_df.columns = failedBank_df.columns.str.strip()

In [72]:
# convert all columns to strings
failedBank_df['Cert'] = failedBank_df['Cert'].astype(str)
failedBank_df['Closing Date'] = failedBank_df['Closing Date'].astype(str)
failedBank_df['Fund'] = failedBank_df['Fund'].astype(str)

## Describe Data
In this section we need to create a JSON describing the features of our data.

In [77]:
failedBank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Bank Name              569 non-null    object
 1   City                   569 non-null    object
 2   State                  569 non-null    object
 3   Cert                   569 non-null    object
 4   Acquiring Institution  569 non-null    object
 5   Closing Date           569 non-null    object
 6   Fund                   569 non-null    object
dtypes: object(7)
memory usage: 31.2+ KB


In [79]:
failedBank_df.head()

,Bank Name,City,State,Cert,Acquiring Institution,Closing Date,Fund
0,Republic First Bank dba Republic Bank,Philadelphia,PA,27332,"Fulton Bank, National Association",26-Apr-24,10546
1,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,3-Nov-23,10545
2,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.",28-Jul-23,10544
3,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.",1-May-23,10543
4,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.",12-Mar-23,10540


In [80]:
# include general_description for as a high-level overview of the data
DATA_DESCRIPTION = {
    'general_description': 'This dataset provides a list of failed banks in the United States.',
    'Bank Name': 'Name of the failed bank.',
    'City': 'City where the failed bank was headquartered.',
    'State': 'State where the failed bank was headquartered.',
    'Cert': 'FDIC certificate number of the failed bank.',
    'Acquiring Institution': 'Name of the institution that acquired the failed bank.',
    'Closing Date': 'Date that the failed bank closed.',
    'Fund': 'FDIC fund number of the failed bank, which acts as a unique identifier in this data set.',
}

## Analyze Tabular Data with an LLM

In [81]:
# read in OPENAI_API_KEY from .env file
import os
from dotenv import load_dotenv
load_dotenv()


True

In [82]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [83]:
# instantiate llm, which defaults to OpenAI GPT-4o
llm = LLM()

In [84]:
# pass data and descriptions to LLM
disc = Discovery(llm=llm, user_input=DATA_DESCRIPTION, data=failedBank_df)
disc.run()

'Based on the provided summary of the dataset, here is a preliminary analysis:\n\n### Overall Details:\n1. **Data Size and Structure**:\n   - The dataset contains 569 entries and 7 columns.\n   - All columns are of type `object`, which means they are treated as strings in pandas.\n\n2. **Missing Values**:\n   - There are no missing values in the dataset, as all columns have 569 non-null entries.\n\n3. **Unique Values**:\n   - The dataset has a mix of columns with high and low cardinality (number of unique values).\n\n### Feature Analysis:\n1. **Bank Name**:\n   - **Count**: 569\n   - **Unique**: 551\n   - **Top**: "The First State Bank" (appears 3 times)\n   - **Observation**: Most bank names are unique, with a few banks appearing more than once.\n\n2. **City**:\n   - **Count**: 569\n   - **Unique**: 436\n   - **Top**: "Chicago" (appears 20 times)\n   - **Observation**: The dataset includes banks from a wide range of cities, with Chicago being the most common.\n\n3. **State**:\n   - **

In [85]:
# function to print markdown
from IPython.display import display, Markdown
def print_markdown(text):
    display(Markdown(text))

In [86]:
print_markdown(disc.discovery)

Based on the provided summary of the dataset, here is a preliminary analysis:

### Overall Details:
1. **Data Size and Structure**:
   - The dataset contains 569 entries and 7 columns.
   - All columns are of type `object`, which means they are treated as strings in pandas.

2. **Missing Values**:
   - There are no missing values in the dataset, as all columns have 569 non-null entries.

3. **Unique Values**:
   - The dataset has a mix of columns with high and low cardinality (number of unique values).

### Feature Analysis:
1. **Bank Name**:
   - **Count**: 569
   - **Unique**: 551
   - **Top**: "The First State Bank" (appears 3 times)
   - **Observation**: Most bank names are unique, with a few banks appearing more than once.

2. **City**:
   - **Count**: 569
   - **Unique**: 436
   - **Top**: "Chicago" (appears 20 times)
   - **Observation**: The dataset includes banks from a wide range of cities, with Chicago being the most common.

3. **State**:
   - **Count**: 569
   - **Unique**: 44
   - **Top**: "GA" (Georgia, appears 93 times)
   - **Observation**: Banks are spread across 44 states, with Georgia having the highest number of failed banks.

4. **Cert**:
   - **Count**: 569
   - **Unique**: 569
   - **Top**: "27332" (appears once)
   - **Observation**: Each bank has a unique FDIC certificate number, making this a unique identifier.

5. **Acquiring Institution**:
   - **Count**: 569
   - **Unique**: 303
   - **Top**: "No Acquirer" (appears 31 times)
   - **Observation**: There are 303 unique acquiring institutions, with some banks not being acquired ("No Acquirer").

6. **Closing Date**:
   - **Count**: 569
   - **Unique**: 264
   - **Top**: "30-Oct-09" (appears 9 times)
   - **Observation**: The closing dates are spread across 264 unique dates, with some dates having multiple bank closures.

7. **Fund**:
   - **Count**: 569
   - **Unique**: 569
   - **Top**: "10546" (appears once)
   - **Observation**: Each bank has a unique FDIC fund number, making this another unique identifier.

### Important Features:
1. **Cert** and **Fund**:
   - Both columns have unique values for each entry, making them important for uniquely identifying each bank.

2. **Bank Name**:
   - While not unique, the bank name is a critical feature for identifying and referencing banks.

3. **State** and **City**:
   - These features provide geographical context, which can be important for regional analysis of bank failures.

4. **Acquiring Institution**:
   - This feature is crucial for understanding the aftermath of bank failures and which institutions are involved in acquisitions.

5. **Closing Date**:
   - The closing date is important for temporal analysis, helping to identify trends over time.

### Summary:
- The dataset is well-structured with no missing values.
- It includes a mix of unique identifiers (Cert, Fund) and categorical data (Bank Name, City, State, Acquiring Institution, Closing Date).
- Key features for analysis include Cert, Fund, Bank Name, State, City, Acquiring Institution, and Closing Date.
- The data allows for various types of analysis, including temporal trends, geographical distribution, and acquisition patterns.

This preliminary analysis provides a foundation for further exploration and modeling, including the potential development of a graph data model.

## Create Initial Data Model via LLM

In [88]:
# instantiate graph data modeler 
gdm = GraphDataModeler(llm=llm, discovery=disc)

# generate model 
gdm.create_initial_model()

ValidationError: 14 validation errors for DataModel
nodes.0.properties.0.csv_mapping.str
  Input should be a valid string [type=string_type, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/string_type
nodes.0.properties.0.csv_mapping.list[str]
  Input should be a valid array [type=list_type, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/list_type
nodes.0.properties.1.csv_mapping.str
  Input should be a valid string [type=string_type, input_value=3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/string_type
nodes.0.properties.1.csv_mapping.list[str]
  Input should be a valid array [type=list_type, input_value=3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/list_type
nodes.1.properties.0.csv_mapping.str
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further informatio